In [1]:
import sys
import os
sys.path.append('site-packages') # manually put all packages/libraries into this folder
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras import models
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering
from scene_cluster_io import *

Using TensorFlow backend.


In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
film = 'extremely_wicked'
frame_choice = list(range(650, 1050))
threshold = 3000

dialogue_folder = os.path.join('dialogue_frames', film)
print('There are', len(os.listdir(dialogue_folder)), 'images in the folder')
print('Selected', len(frame_choice), 'of those frames')

hac_labels = label_clusters(dialogue_folder, frame_choice, film, threshold)

There are 6603 images in the folder
Selected 400 of those frames
Number of clusters: 18


In [4]:
tuned_model = models.load_model('saved_models/tuned_model')

In [5]:
y_pred_values = predict_mcu(dialogue_folder, tuned_model, frame_choice, film)

In [6]:
shot_id_list = get_shot_ids(frame_choice, hac_labels)

In [7]:
scene_df = pd.DataFrame(zip(frame_choice, hac_labels, shot_id_list, y_pred_values), columns=['frame_file', 'cluster', 'shot_id', 'mcu'])
scene_df.head(3)

,frame_file,cluster,shot_id,mcu
0,650,2,0,1
1,651,2,0,1
2,652,2,0,1


In [8]:
alternating_pairs = get_alternating_pairs(frame_choice, hac_labels, y_pred_values, shot_id_list)
alternating_pairs

[[0, 2], [9, 13], [0, 1], [5, 10], [0, 6]]

In [9]:
speaker_pairs = mcu_check(alternating_pairs, scene_df)
speaker_pairs

cluster	 count	 mcu probability
0 	 165 	 48.48%
2 	 61 	 65.57%
Fails MCU check

9 	 28 	 96.43%
13 	 21 	 100.00%
Passes MCU check

0 	 165 	 48.48%
1 	 9 	 66.67%
Fails MCU check

5 	 23 	 95.65%
10 	 17 	 100.00%
Passes MCU check

0 	 165 	 48.48%
6 	 9 	 55.56%
Fails MCU check



[[9, 13], [5, 10]]

In [10]:
anchors = anchor_scenes(speaker_pairs, scene_df)
anchors

Speaker A and B Clusters: [9, 13]
Anchor Start/End Frames: 690 743

Speaker A and B Clusters: [5, 10]
Anchor Start/End Frames: 954 997



[(690, 743), (954, 997)]

In [11]:
scenes = expand_scenes(speaker_pairs, scene_df)
scenes

Speaker A and B Clusters: [9, 13]
Anchor Start/End Frames: 690 743
Cutaway Clusters: [0]
Expanded Start/End Frames: 690 745

Speaker A and B Clusters: [5, 10]
Anchor Start/End Frames: 954 997
Cutaway Clusters: [16]
Expanded Start/End Frames: 954 997



[(690, 745), (954, 997)]

In [14]:
def generate_scenes(film, frame_choice, threshold):
    dialogue_folder = os.path.join('dialogue_frames', film)

    hac_labels = label_clusters(dialogue_folder, frame_choice, film, threshold)
    print()
    y_pred_values = predict_mcu(dialogue_folder, tuned_model, frame_choice, film)
    shot_id_list = get_shot_ids(frame_choice, hac_labels)
    scene_df = pd.DataFrame(zip(frame_choice, hac_labels, shot_id_list, y_pred_values), columns=['frame_file', 'cluster', 'shot_id', 'mcu'])
    alternating_pairs = get_alternating_pairs(frame_choice, hac_labels, y_pred_values, shot_id_list)
    speaker_pairs = mcu_check(alternating_pairs, scene_df)
    anchors = anchor_scenes(speaker_pairs, scene_df)
    scenes = expand_scenes(speaker_pairs, scene_df)

    return scenes

In [13]:
def clean_scene_list(scene_list):

    # take list of lists and just make a singular list
    scene_set = []

    for results in scene_list:
        for scene in results:
            scene_set.append(scene)

    # remove duplicates while maintaining order
    seen = set()
    seen_add = seen.add
    
    return [x for x in scene_set if not (x in seen or seen_add(x))]

In [6]:
tuned_model = models.load_model('saved_models/tuned_model')

In [15]:
film = 'hustle'
threshold = 3000

x = 200

frame_choices = []

#dialogue_folder = os.path.join('dialogue_frames', film)
#while x < (len(os.listdir(dialogue_folder)) - 200):
while x < (2000 - 200):
    frame_choices.append(range(x - 200, x + 200))
    x += 100

scene_list = []

for frames in frame_choices:
    scene_list.append(generate_scenes(film, frames, threshold))

scene_list = clean_scene_list(scene_list)

Number of clusters: 32

cluster	 count	 mcu probability
1 	 7 	 28.57%
8 	 28 	 7.14%
Fails MCU check

8 	 28 	 7.14%
26 	 20 	 45.00%
Fails MCU check

6 	 13 	 38.46%
11 	 27 	 37.04%
Fails MCU check

11 	 27 	 37.04%
14 	 68 	 16.18%
Fails MCU check

Number of clusters: 31

cluster	 count	 mcu probability
0 	 38 	 89.47%
19 	 16 	 100.00%
Passes MCU check

0 	 38 	 89.47%
6 	 19 	 100.00%
Passes MCU check

3 	 7 	 28.57%
7 	 36 	 8.33%
Fails MCU check

7 	 36 	 8.33%
11 	 20 	 45.00%
Fails MCU check

4 	 13 	 38.46%
14 	 27 	 37.04%
Fails MCU check

2 	 56 	 16.07%
14 	 27 	 37.04%
Fails MCU check

5 	 22 	 0.00%
22 	 9 	 0.00%
Fails MCU check

13 	 23 	 95.65%
17 	 7 	 71.43%
Passes MCU check

13 	 23 	 95.65%
27 	 7 	 100.00%
Passes MCU check

Speaker A and B Clusters: [0, 19]
Anchor Start/End Frames: 100 224

Speaker A and B Clusters: [0, 6]
Anchor Start/End Frames: 100 224

Speaker A and B Clusters: [13, 17]
Anchor Start/End Frames: 451 488

Speaker A and B Clusters: [13, 27]
Anc

Number of clusters: 36

cluster	 count	 mcu probability
1 	 26 	 69.23%
6 	 12 	 91.67%
Passes MCU check

0 	 14 	 28.57%
9 	 11 	 100.00%
Fails MCU check

7 	 28 	 89.29%
11 	 23 	 100.00%
Passes MCU check

10 	 26 	 96.15%
11 	 23 	 100.00%
Passes MCU check

5 	 35 	 94.29%
11 	 23 	 100.00%
Passes MCU check

Speaker A and B Clusters: [1, 6]
Anchor Start/End Frames: 1332 1409

Speaker A and B Clusters: [7, 11]
Anchor Start/End Frames: 1410 1544

Speaker A and B Clusters: [10, 11]
Anchor Start/End Frames: 1410 1544

Speaker A and B Clusters: [5, 11]
Anchor Start/End Frames: 1410 1544

Speaker A and B Clusters: [1, 6]
Anchor Start/End Frames: 1332 1409
Cutaway Clusters: [22 16  0 25  2 27 19]
Expanded Start/End Frames: 1330 1409

Speaker A and B Clusters: [7, 11]
Anchor Start/End Frames: 1410 1544
Cutaway Clusters: [ 0  9 14  5 10]
Expanded Start/End Frames: 1410 1544

Speaker A and B Clusters: [10, 11]
Anchor Start/End Frames: 1410 1544
Cutaway Clusters: [ 0  9 14  7  5]
Expanded Star

In [16]:
scene_list

[(100, 225),
 (449, 493),
 (200, 224),
 (449, 542),
 (648, 760),
 (543, 796),
 (509, 579),
 (622, 899),
 (814, 917),
 (712, 757),
 (758, 933),
 (1054, 1399),
 (1330, 1409),
 (1410, 1499),
 (1410, 1544),
 (1327, 1692),
 (1403, 1544),
 (1400, 1799),
 (1653, 1736),
 (1500, 1544),
 (1798, 1839)]

In [ ]:
# this will work for an entire movie but runs out of memory
# maybe just do a quarter at a time?
film = 'hustle'
threshold = 3000

x = 200

frame_choices = []
dialogue_folder = os.path.join('dialogue_frames', film)

while x < (len(os.listdir(dialogue_folder)) - 200):
    frame_choices.append(range(x - 200, x + 200))
    x += 100

scene_list = []

for frames in frame_choices:
    scene_list.append(generate_scenes(film, list(frames), threshold))

scene_list = clean_scene_list(scene_list)